In [ ]:
#importing libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.svm import SVC

%matplotlib inline


In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
df = pd.read_csv('root2ai - Data.csv')
df

,Text,Target
0,reserve bank forming expert committee based in...,Blockchain
1,director could play role financial system,Blockchain
2,preliminary discuss secure transaction study r...,Blockchain
3,security indeed prove essential transforming f...,Blockchain
4,bank settlement normally take three days based...,Blockchain
...,...,...
22699,fourth study discusses blockchain technology e...,Reg Tech
22700,book finishes stating biggest issue emerging F...,Reg Tech
22701,people culture cess,Reg Tech
22702,author challenges execu tive lead change stop ...,Reg Tech


In [ ]:
k = df.Target.value_counts().keys()
k[0]

'FinTech'

In [ ]:
d = {}
for i in range(len(k)):
    if i<5:
        d[k[i]] = i
    else :
        d[k[i]]=5
df.Target = df.Target.replace(d)
df.Target

0        4.0
1        4.0
2        4.0
3        4.0
4        4.0
        ... 
12951    0.0
12952    0.0
12953    0.0
12954    0.0
12955    NaN
Name: Target, Length: 12956, dtype: float64

In [ ]:
import re
nltk.download('stopwords')
s = stopwords.words('english')
s.extend(['today', 'tomorrow', 'outside', 'out', 'there'])
ps = nltk.wordnet.WordNetLemmatizer()
for i in range(df.shape[0]):
    review = str(df.loc[i,'Text'])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    df.loc[i, 'Text'] = review


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
X_train1 = df['Text']
y_train_vec = df['Target']

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
X = [i.split(' ') for i in X_train1]
model = Word2Vec(sentences=X, size=100, window=5, min_count=1, workers=4)
word_vectors = model.wv
word_vectors

In [ ]:
X_train_vec = []
for line in X_train1:
    vec = [word_vectors[i] for i in line.split(' ')]
    vec = np.mean(vec,axis=-1)
    if not vec.shape:
        vec = [0,0]
    vec = np.mean(vec)
#     vec = np.expand_dims(vec,-1)
#     vec = np.expand_dims(vec)
#     vec = vec.reshape(-1,1)
    X_train_vec.append(vec)

In [ ]:
# X_train_vec = np.array(X_train_vec)
X_train_vec =np.expand_dims(X_train_vec,-1)

In [ ]:
X_train_vec[0]

array([0.04631805], dtype=float32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split((X_train_vec), y_train_vec, test_size=0.33, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
neigh.score(X_test,y_test)

0.2854664353396503

In [ ]:
#predicting the test set result
y_pred = neigh.predict(X_test)
#check accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

Accuracy: 0.29


In [ ]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

                  precision    recall  f1-score   support

         Bigdata       0.40      0.40      0.40       761
      Blockchain       0.10      0.11      0.11       439
  Cyber Security       0.19      0.20      0.19       862
   Data Security       0.04      0.03      0.04       115
         FinTech       0.42      0.42      0.42      2852
   Microservices       0.07      0.06      0.06       352
        Neobanks       0.17      0.17      0.17       347
        Reg Tech       0.32      0.36      0.34       699
   Robo Advising       0.03      0.03      0.03       249
   Stock Trading       0.05      0.04      0.04       251
credit reporting       0.13      0.12      0.12       566

        accuracy                           0.29      7493
       macro avg       0.17      0.18      0.18      7493
    weighted avg       0.28      0.29      0.28      7493

